In [1]:
import netCDF4 as nc

In [2]:
import numpy as np

x = np.linspace(-6371000, 6371000, 100)
y = x.copy()
z = x.copy()

X, Y, Z = np.meshgrid(x, y, z)

# Calculate the distance from the center for each point
distance_from_center = np.sqrt(X**2 + Y**2 + Z**2)

# Create the VP, VS, and RHO arrays
normalized_X = X / 6371000
normalized_Y = Y / 6371000
normalized_Z = Z / 6371000
x_coord, y_coord, z_coord = (0.5, 0, 0)
x_coord2, y_coord2, z_coord2 = (0, 0.5, 0)
x_coord3, y_coord3, z_coord3 = (0, 0, -0.5)
sigma = 0.3
VP = np.exp(-(((normalized_X - x_coord)/sigma)**2 +
              ((normalized_Y - y_coord)/sigma)**2 +
              ((normalized_Z - z_coord)/sigma)**2)) + \
              np.exp(-(((normalized_X - x_coord2)/sigma)**2 +
              ((normalized_Y - y_coord2)/sigma)**2 +
              ((normalized_Z - z_coord2)/sigma)**2)) - \
              np.exp(-(((normalized_X - x_coord3)/sigma)**2 +
              ((normalized_Y - y_coord3)/sigma)**2 +
              ((normalized_Z - z_coord3)/sigma)**2))

VS = np.ones(X.shape)
RHO = np.ones(X.shape)

# Set the values to 0 where the distance from the center is greater than 6371000
VP[distance_from_center > 6371000] = 0
VS[distance_from_center > 6371000] = 0
RHO[distance_from_center > 6371000] = 0


radius = np.sqrt(X**2 + Y**2 + Z**2)
latitude = np.arcsin(Z / radius)  # in radians
longitude = np.arctan2(Y, X)  # in radians


In [3]:
from mayavi import mlab

# Create a new figure
mlab.figure(size=(600, 600))

# Create a scalar field
src = mlab.pipeline.scalar_field(VP)

# Create a cut plane through the scalar field
cut_plane = mlab.pipeline.scalar_cut_plane(src, plane_orientation='z_axes')

# Set the view
mlab.view(azimuth=0, elevation=90)

# Show the result
mlab.show()

In [4]:
import netCDF4 as nc
import numpy as np

# Create a new NetCDF file
rootgrp = nc.Dataset("example.nc", "w", format="NETCDF4")

# Define the dimensions
dim_radius = rootgrp.createDimension("dim_radius", 191)
dim_latitude = rootgrp.createDimension("dim_latitude", 676)
dim_longitude = rootgrp.createDimension("dim_longitude", 676)

# Define the variables
radius = rootgrp.createVariable("radius", "f4", ("dim_radius",))
latitude = rootgrp.createVariable("latitude", "f4", ("dim_latitude",))
longitude = rootgrp.createVariable("longitude", "f4", ("dim_longitude",))
VP = rootgrp.createVariable("VP", "f4", ("dim_radius", "dim_latitude", "dim_longitude"))
VS = rootgrp.createVariable("VS", "f4", ("dim_radius", "dim_latitude", "dim_longitude"))
RHO = rootgrp.createVariable("RHO", "f4", ("dim_radius", "dim_latitude", "dim_longitude"))

# Assign values to the variables
radius[:] = np.arange(191)
latitude[:] = np.arange(676)
longitude[:] = np.arange(676)
VP[:] = np.random.rand(191, 676, 676)
VS[:] = np.random.rand(191, 676, 676)
RHO[:] = np.random.rand(191, 676, 676)

# Close the NetCDF file
rootgrp.close()